## Homework

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

## Install packages

In [ ]:
!pip install pandas
!pip install pyarrow
!pip install scikit-learn

## Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2022.

In [106]:
!curl -o taxi_trips_january.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet
!curl -o taxi_trips_february.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  2 36.3M    2  831k    0     0  1193k      0  0:00:31 --:--:--  0:00:31 1194k
 69 36.3M   69 25.1M    0     0  14.7M      0  0:00:02  0:00:01  0:00:01 14.7M
100 36.3M  100 36.3M    0     0  18.4M      0  0:00:01  0:00:01 --:--:-- 18.4M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  3 43.5M    3 1675k    0     0  2414k      0  0:00:18 --:--:--  0:00:18 2414k
 69 43.5M   69 30.2M    0     0  17.8M      0  0:00:02  0:00:01  0:00:01 17.8M
100 43.5M  100 43.5M    0     0  18.2M      0  0:00:02  0:00:02 --:--:-- 18.2M


## Reading data

In [51]:
!python -V

Python 3.11.2


In [31]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [32]:
def read(path:str)->pd.DataFrame:
    "Load a parquet object from the file path, returning a DataFrame."
    return pd.read_parquet(path)

def prepare(df:pd.DataFrame)->pd.DataFrame:
    "Prepare a raw dataframe, returning a prepare DataFrame"
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] =  df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    return df

def train(df_train:pd.DataFrame, df_test:pd.DataFrame):
    "Train a Linear Regression and calculate the RMSE on the validation dataframe"
    categorical = ['PULocationID', 'DOLocationID']
    target = 'duration'
    
    df_train[categorical] = df_train[categorical].astype(str)
    df_test[categorical] = df_test[categorical].astype(str)   
    
    dv = DictVectorizer()   
    train_dicts = df_train[categorical].to_dict(orient='records')
    test_dicts = df_test[categorical].to_dict(orient='records')
    
    X_train = dv.fit_transform(train_dicts)
    X_test = dv.transform(test_dicts)
    
    y_train = df_train[target].values
    y_test = df_test[target].values

    lr = LinearRegression()
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_test)

    mse =mean_squared_error(y_test, y_pred, squared=False)
    
    return dv,mse 
    
    

In [8]:

raw_january = read('taxi_trips_january.parquet') 

In [9]:
raw_january.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


## Q1. Downloading the data

Read the data for January. How many columns are there?



In [11]:
print(f'There are {len(raw_january.columns)} columns in data for january')

There are 19 columns in data for january


## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

In [12]:
processed_january = prepare(raw_january)

In [13]:
std_trips_duration =  processed_january['duration'].std()
print(f'The standard deviation of trips in January is {std_trips_duration}')

The standard deviation of trips in January is 8.99928164030397


## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [15]:
print(len(processed_january)/len(raw_january))

0.9827547930522406


The fraction of records left after drop outliers is 98%

## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [17]:
dv,mse = train(processed_january,processed_january)

C:\Users\user\AppData\Local\Temp\ipykernel_11576\3341645341.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[categorical] = df_train[categorical].astype(str)


In [18]:
len(dv.feature_names_)

515

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters 
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [19]:
mse

6.986190841034851

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2022). 

What's the RMSE on validation?

* 7.79
* 12.79
* 17.79
* 22.79

In [26]:
raw_february = read('taxi_trips_february.parquet') 
processed_february = prepare(raw_february)

In [29]:
dv,mse = train(processed_january,processed_february)

C:\Users\user\AppData\Local\Temp\ipykernel_11576\4235317958.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[categorical] = df_train[categorical].astype(str)
C:\Users\user\AppData\Local\Temp\ipykernel_11576\4235317958.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[categorical] = df_test[categorical].astype(str)


In [30]:
mse

7.786408695263233